In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import os
input_fname = 'test_IOTA.in'
np.set_printoptions(precision=3)

# Prepare Beam and Lattice

In [2]:
beam, lattice0 = impact.readInputFile(input_fname)

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [3]:
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=10
beam.current=0.0
dist = beam.distribution
dist.distribution_type = 'Waterbag'
lattice = [impact.getElem('loop'),
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl')]
lattice[0].nturn = 1
lattice[1].file_id = 1001
lattice[3].file_id = 1002
lattice[5].file_id = 1003


ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

In [4]:
dist.mode = 'twiss'
dist.betx = 10.0
dist.alfx = 0
dist.emitx = 1.0e-6
dist.bety = 10.0
dist.alfy = 0
dist.emity = 1.0e-6
dist.betz = 109.0
dist.alfz = 0
dist.emitz = 1.0e-4

# 1. Drift

In [5]:
elemF = impact.getElem('drift')
elemF.length = 1.1
elemF.n_sckick = 20
elemF.n_map = 2

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [6]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00 -8.327e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  6.939e-18  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00 -3.469e-18  0.000e+00]
 [ 0.000e+00  0.000e+00  8.132e-20  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]]


# 2. Quad

In [7]:
elemF = impact.getElem('quad')
elemF.length = 0.02
elemF.n_sckick = 20
elemF.n_map = 1
elemF.B1 = 15.0

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [8]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 4.163e-17  2.927e-18 -4.554e-18  2.060e-18 -9.498e-06  0.000e+00]
 [ 0.000e+00  1.030e-18  5.725e-17 -8.890e-18 -4.964e-06  0.000e+00]
 [-3.469e-18  0.000e+00  2.776e-17  4.066e-19 -8.605e-07  0.000e+00]
 [-1.388e-17 -2.602e-18 -1.908e-17  2.819e-18 -1.939e-06  0.000e+00]
 [ 0.000e+00  0.000e+00 -4.554e-18  3.469e-18  6.080e-07  0.000e+00]
 [-8.674e-18 -1.897e-18  3.117e-19 -5.421e-18 -1.563e-06  0.000e+00]
 [-1.301e-18 -1.301e-18  3.296e-17 -2.033e-18 -1.052e-06  0.000e+00]
 [ 0.000e+00  0.000e+00  1.214e-17  2.927e-18 -4.649e-07  0.000e+00]
 [-1.388e-17  6.505e-19 -2.082e-17  0.000e+00 -4.232e-06  0.000e+00]
 [ 0.000e+00  0.000e+00 -9.541e-18 -1.193e-18 -3.658e-06  0.000e+00]]


# 3. Sextupole

In [9]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'multipole_thin':
        break
lattice0[i]


 KL_dipole: 0.0
   KL_quad: 0.0
   KL_sext: 0.15
    KL_oct: 0.0
   KL_deca: 0.0
 KL_dodeca: 0.0
      type: 'multipole_thin'

In [10]:
elemF = lattice0[i]

elemB = copy(elemF)
elemF.KL_sext = 10.0
elemB.KL_sext =-elemF.KL_sext

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [11]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData1[:,:6]-pData0[:,:6])

[[ 0.000e+00 -3.518e-03  0.000e+00  3.806e-04  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.744e-04  0.000e+00  2.252e-03  0.000e+00  0.000e+00]
 [ 0.000e+00  6.291e-05  0.000e+00 -1.052e-03  0.000e+00  0.000e+00]
 [ 0.000e+00 -2.008e-04  0.000e+00 -9.983e-04  0.000e+00  0.000e+00]
 [ 0.000e+00 -8.150e-05  0.000e+00 -6.777e-05  0.000e+00  0.000e+00]
 [ 0.000e+00 -8.940e-04  0.000e+00 -1.494e-05  0.000e+00  0.000e+00]
 [ 0.000e+00  7.295e-04  0.000e+00 -3.694e-04  0.000e+00  0.000e+00]
 [ 0.000e+00  1.748e-04  0.000e+00 -1.090e-04  0.000e+00  0.000e+00]
 [ 0.000e+00 -9.415e-04  0.000e+00 -1.782e-03  0.000e+00  0.000e+00]
 [ 0.000e+00 -4.768e-04  0.000e+00 -9.696e-04  0.000e+00  0.000e+00]]


In [12]:
print(pData2[:,:6]-pData0[:,:6])

[[0.000e+00 2.168e-19 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 1.084e-19 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 1.626e-19 0.000e+00 0.000e+00]]


# 4. Dipole

In [13]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'dipole':
        break
lattice0[i]


                   length: 0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: 1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.058 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: 0.5 [1]
                     type: 'dipole'

### 4.1 FINT=0

##### negative length 

In [14]:
elemF = copy(lattice0[i])
elemF.fringe_field_integration = 0.0
elemF.entrance_angle=0.03
elemF.exit_angle=0.06

elemB = copy(elemF)
elemB.length = -elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [15]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-3.200e-04 -8.982e-04  2.489e-05  2.970e-05  2.274e+01  0.000e+00]
 [ 9.799e-05  7.803e-04 -4.173e-04 -7.304e-05 -1.368e+01  0.000e+00]
 [-3.092e-04  8.929e-05 -3.177e-04 -4.763e-05  9.250e+00  0.000e+00]
 [ 2.517e-04  6.676e-05  2.486e-04  7.047e-05 -9.382e+00  0.000e+00]
 [ 2.993e-04 -4.223e-04  1.685e-05  4.567e-05 -4.488e+00  0.000e+00]
 [-1.571e-04 -4.843e-04  3.054e-05 -4.384e-05  1.174e+01  0.000e+00]
 [ 1.299e-05 -2.911e-04 -3.739e-04 -5.624e-05  3.431e+00  0.000e+00]
 [-1.708e-04  2.751e-04 -1.996e-04 -9.833e-06  2.109e+00  0.000e+00]
 [ 4.374e-04  1.910e-05  3.202e-04  3.578e-05 -1.502e+01  0.000e+00]
 [ 4.659e-05  6.556e-04  2.293e-04  4.056e-05 -1.029e+01  0.000e+00]]


### negative length and bending angle and negative/exchanged e1,e2

In [16]:
elemB.bending_angle = -elemB.bending_angle
elemB.entrance_angle= -elemF.exit_angle
elemB.exit_angle    = -elemF.entrance_angle

impact.writeInputFile(beam,lattice);
impact.run()

0

In [17]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-3.469e-17 -1.355e-17  2.168e-19 -1.084e-19  4.816e-15  0.000e+00]
 [-1.735e-17 -1.865e-17  1.735e-18  5.421e-19  4.635e-15  0.000e+00]
 [-1.735e-18  5.204e-18  0.000e+00  0.000e+00 -8.882e-16  0.000e+00]
 [ 2.602e-17  1.095e-17  0.000e+00 -2.168e-19 -4.989e-15  0.000e+00]
 [-1.041e-17 -6.505e-18  0.000e+00 -2.168e-19  2.179e-15  0.000e+00]
 [-6.939e-18 -4.770e-18  5.421e-20  0.000e+00  5.468e-15  0.000e+00]
 [-6.072e-18 -5.638e-18  0.000e+00  2.711e-19  2.928e-15  0.000e+00]
 [ 1.735e-18  1.084e-18  0.000e+00 -1.084e-19 -5.690e-16  0.000e+00]
 [-1.388e-17 -9.433e-18  0.000e+00  1.355e-19  3.081e-15  0.000e+00]
 [ 2.602e-17  8.240e-18  1.735e-18  0.000e+00 -4.913e-15  0.000e+00]]


### 4.2 FINT != 0, e1=e2=0

In [18]:
elemF = copy(lattice0[i])
elemF.fringe_field_integration = 10.0


elemB = copy(elemF)
elemB.length = -elemF.length
elemB.bending_angle = -elemF.bending_angle
elemB.fringe_field_integration = -elemF.fringe_field_integration


lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [19]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-3.469e-17 -1.063e-17  6.505e-19 -1.626e-18 -6.523e-16  0.000e+00]
 [-1.735e-17 -1.306e-17  0.000e+00  3.253e-19  7.161e-15  0.000e+00]
 [ 1.214e-17  5.204e-18  0.000e+00 -3.849e-18 -5.412e-15  0.000e+00]
 [ 2.776e-17  1.637e-17  1.735e-18 -1.735e-18 -5.836e-15  0.000e+00]
 [-1.041e-17 -9.541e-18  6.505e-19 -8.674e-19  3.497e-15  0.000e+00]
 [-1.214e-17 -1.529e-17 -1.762e-19  0.000e+00  3.969e-15  0.000e+00]
 [-3.036e-18 -6.939e-18  0.000e+00 -9.731e-18  2.123e-15  0.000e+00]
 [ 4.770e-18  4.120e-18 -3.469e-18  3.361e-18 -1.443e-15  0.000e+00]
 [-1.388e-17 -5.475e-18  5.204e-18 -5.909e-18  5.163e-15  0.000e+00]
 [ 1.735e-17  1.648e-17  1.735e-18 -3.957e-18 -2.970e-15  0.000e+00]]


### negative length, bending angle and FINT

###### MaryLie_order = 2

elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
os.system('xmain_dipoleF2')
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

###### MaryLie_order = 3

elemB.bending_angle = -elemF.bending_angle
elemF.fringe_field_integration = 10000000.0
elemB.fringe_field_integration = -elemF.fringe_field_integration
impact.writeInputFile(beam,lattice);
os.system('xmain_dipoleF3')
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

### 4.2 FINT != 0, e1,e2!=0

In [20]:
elemF = copy(lattice0[i])
elemF.fringe_field_integration = 10.0
elemF.entrance_angle=0.05
elemF.exit_angle=0.05

elemB = copy(elemF)
elemB.length = -elemF.length
elemB.bending_angle = -elemF.bending_angle
elemB.fringe_field_integration = -elemF.fringe_field_integration
elemB.entrance_angle = -elemF.exit_angle
elemB.exit_angle     = -elemF.entrance_angle

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [21]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[-1.735e-17  7.589e-19  2.168e-19 -3.253e-19  7.303e-15  0.000e+00]
 [-1.735e-17 -9.378e-18 -5.204e-18  1.225e-17  6.189e-15  0.000e+00]
 [ 1.214e-17  6.722e-18  3.469e-18 -1.274e-17 -4.940e-15  0.000e+00]
 [ 2.949e-17  1.626e-17  0.000e+00  2.819e-18 -7.078e-15  0.000e+00]
 [-8.674e-18 -8.240e-18  0.000e+00  0.000e+00  2.501e-15  0.000e+00]
 [-8.674e-18 -1.615e-17  5.421e-20 -1.301e-18  3.664e-15  0.000e+00]
 [-6.072e-18 -1.301e-18  3.469e-18 -3.849e-18  1.263e-15  0.000e+00]
 [ 4.770e-18  2.602e-18  0.000e+00  3.253e-19 -1.138e-15  0.000e+00]
 [-1.388e-17 -1.149e-17  0.000e+00  0.000e+00  3.664e-15  0.000e+00]
 [ 2.255e-17  5.421e-18  0.000e+00 -2.439e-18 -3.997e-15  0.000e+00]]


# 5. Nonlinear Insert

In [22]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'nonlinear_insert':
        break
lattice0[i]


             length: 1.8 [m]
           n_sckick: 96 [1]
              n_map: 5 [1]
         strength_t: 0.4
 transverse_scale_c: 0.01
        pipe_radius: 10.0 [m]
       tune_advance: 0.3034496449165134
               type: 'nonlinear_insert'

In [23]:
elemF = lattice0[i]

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [24]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 3.123e-17 -1.626e-18  1.518e-18 -1.735e-18 -8.327e-17  0.000e+00]
 [ 1.735e-17 -2.223e-18  1.561e-17  6.180e-18 -2.220e-16  0.000e+00]
 [-2.082e-17 -1.670e-17  2.602e-17  1.054e-17  4.441e-16  0.000e+00]
 [-2.082e-17  1.149e-17 -1.735e-18  4.337e-19  3.053e-16 -1.137e-13]
 [ 6.939e-17 -1.327e-16 -7.481e-17  6.787e-17  1.284e-16  0.000e+00]
 [ 2.446e-16 -4.627e-15 -5.398e-16  2.115e-14  4.163e-16  0.000e+00]
 [-4.337e-18 -1.301e-18 -5.204e-18  2.467e-18 -5.551e-17  0.000e+00]
 [-9.758e-18  1.084e-18 -1.821e-17  8.890e-18  5.690e-16  0.000e+00]
 [ 0.000e+00  3.144e-18 -5.204e-18  2.331e-18  2.776e-16  0.000e+00]
 [ 0.000e+00  1.106e-17 -1.735e-17  1.409e-18 -1.110e-16  0.000e+00]]


# 6. Linear matrix map (toy model compoenent)

In [25]:
elemF = impact.getElem('linear_matrix_map')
elemF.nonlinear_insert_length = 1.8
elemF.nonlinear_insert_tuneAdvance = 0.3034496449165134
elemF.tune_advance = 0.22

elemB = copy(elemF)
elemB.tune_advance =-elemF.tune_advance
elemB.nonlinear_insert_tuneAdvance =-elemF.nonlinear_insert_tuneAdvance
elemB.nonlinear_insert_length =-elemF.nonlinear_insert_length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [26]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 0.000e+00 -9.758e-19  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  5.421e-19  0.000e+00  3.253e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -8.132e-19  0.000e+00  0.000e+00]
 [ 0.000e+00 -8.674e-19  0.000e+00 -2.168e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -2.168e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  2.711e-19  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -8.403e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  3.253e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  1.518e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  4.337e-19  0.000e+00  0.000e+00]]


# 7. ToyModel

In [27]:
elemF = impact.getElem('nonlinear_insert')
elemF.tune_advance = 0.3034496449165134
elemF.length = 1.8
elemF.n_sckick = 100
elemF.n_map = 10
elemF.strength_t = 0.4
elemF.transverse_scale_c = 0.01

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

### tune advance error = 0

In [28]:
elemF1 = impact.getElem('linear_matrix_map')
elemF1.nonlinear_insert_length = 1.8
elemF1.nonlinear_insert_tuneAdvance = 0.3034496449165134
elemF1.tune_advance_x = 0.05
elemF1.tune_advance_y = 0.02

elemB1 = copy(elemF1)
elemB1.tune_advance_x =-elemB1.tune_advance_x
elemB1.tune_advance_y =-elemB1.tune_advance_y
elemB1.nonlinear_insert_tuneAdvance =-elemB1.nonlinear_insert_tuneAdvance
elemB1.nonlinear_insert_length =-elemB1.nonlinear_insert_length

lattice.insert(3,elemF1)
lattice.insert(5,elemB1)

In [29]:
for i in range(len(lattice)):
    print(lattice[i].type)

loop
write_raw_ptcl
nonlinear_insert
linear_matrix_map
write_raw_ptcl
linear_matrix_map
nonlinear_insert
write_raw_ptcl


In [30]:
impact.writeInputFile(beam,lattice);
impact.run()

0

In [31]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:,:6]-pData2[:,:6])

[[ 1.388e-17  1.225e-17  3.469e-18 -8.999e-18  7.286e-17  0.000e+00]
 [ 0.000e+00  1.193e-18 -3.296e-17  8.674e-18  3.886e-16  0.000e+00]
 [ 8.674e-17 -1.128e-17 -6.939e-18 -1.892e-17  4.441e-16  0.000e+00]
 [-5.725e-17  3.036e-17 -3.990e-17  2.776e-17 -1.388e-16 -1.137e-13]
 [ 8.066e-17 -1.596e-16 -9.519e-17  9.996e-17  1.978e-16  0.000e+00]
 [ 3.365e-16 -6.349e-15 -7.578e-16  2.973e-14  1.388e-15  0.000e+00]
 [-2.689e-17  2.342e-17  1.735e-17 -1.626e-18  0.000e+00  0.000e+00]
 [-3.036e-18  8.240e-18 -2.602e-18  2.559e-17  9.714e-17  0.000e+00]
 [-1.388e-17  5.421e-19  3.469e-18 -1.789e-18 -3.331e-16  0.000e+00]
 [ 6.592e-17  9.758e-18  3.730e-17 -4.424e-17 -3.331e-16  0.000e+00]]
